In [46]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from helper.modules import EpilepticSeizure
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Input, LSTM, Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout, Concatenate, Resizing


### Function to load and preprocess images from paths

In [5]:

def load_and_preprocess_image(image_path, target_size=(64, 64)):
    image = load_img(image_path, target_size=target_size, color_mode='rgb')
    image = img_to_array(image)
    image = image / 255.0  # Normalize to [0, 1] range
    return image


### Generating Dataset

In [6]:
config = {
    "data_path": "Data/",
    "preprocess": True,
    "generate_images": True,
    "rows_per_person": 23,
    "frequency": 178,
    "labels": 3,
    "add_noise": True,
    "oversampling": True,
    "undersampling": True,
    "test_size": 0.3
}

es = EpilepticSeizure(config)

X_train, y_train, X_test, y_test = es.process()

X_train.shape, y_train.shape, X_test.shape, y_test.shape

Processing Patients Test: 100%|██████████| 150/150 [00:05<00:00, 27.33it/s]


((30728, 179), (30728, 1), (3450, 179), (3450, 1))

### Data Structure

In [33]:
X_train


(30728, 179)

In [18]:
X_train_signals = X_train.iloc[:, :178].values
X_train_signals = X_train_signals.reshape((X_train_signals.shape[0], 1, X_train_signals.shape[1]))

X_test_signals = X_test.iloc[:, :178].values
X_test_signals = X_test_signals.reshape((X_test_signals.shape[0], 1, X_test_signals.shape[1]))

X_train_images = np.array([load_and_preprocess_image(path) for path in tqdm(X_train['Image_Path'],desc="Processing train images")])
X_test_images = np.array([load_and_preprocess_image(path) for path in tqdm(X_test['Image_Path'],desc="Processing test images")])

# Labels
y_train = y_train['Label'].values
y_test = y_test['Label'].values

Processing train images:   0%|          | 0/30728 [00:00<?, ?it/s]

Processing test images: 100%|██████████| 3450/3450 [00:11<00:00, 294.42it/s]


In [34]:
X_train_images.shape

(30728, 64, 64, 3)

### Defining CNN Model

In [48]:
CNN_model = Sequential()

new_image_height = 64
new_image_width = 64

CNN_model.add(Resizing(new_image_height, new_image_width, input_shape=(64,64, 3)))

CNN_model.add(Conv2D(32, kernel_size=(1, 1), activation='relu'))
CNN_model.add(BatchNormalization())
CNN_model.add(MaxPooling2D(pool_size=(1, 1)))

CNN_model.add(Conv2D(64, kernel_size=(1, 1), activation='relu'))
CNN_model.add(BatchNormalization())
CNN_model.add(MaxPooling2D(pool_size=(1, 1)))

CNN_model.add(Conv2D(128, kernel_size=(1, 1), activation='relu'))
CNN_model.add(BatchNormalization())
CNN_model.add(MaxPooling2D(pool_size=(1, 1)))

CNN_model.add(Flatten())

CNN_model.add(Dense(128, activation='relu'))
CNN_model.add(BatchNormalization())
CNN_model.add(Dropout(0.5))


CNN_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


c:\Python312\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:18: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


### Defining LSTM Model

In [50]:
LSTM_model = Sequential()
LSTM_model.add(LSTM(256, input_shape=(1, 178), activation='relu', return_sequences=True))
LSTM_model.add(LSTM(128, activation='relu', return_sequences=True))
LSTM_model.add(LSTM(64, activation='relu', return_sequences=True))
LSTM_model.add(LSTM(32, activation='relu'))
LSTM_model.add(Dropout(0.5))

LSTM_model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_12 (LSTM)                  │ (None, 1, 256)         │       445,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_13 (LSTM)                  │ (None, 1, 128)         │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_14 (LSTM)                  │ (None, 1, 64)          │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_15 (LSTM)                  │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 32)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 704,384 (2.69 MB)

 Trainable params: 704,384 (2.69 MB)

 Non-trainable params: 0 (0.00 B)

### Defining Model Inputs

In [51]:
eeg_input = Input(shape=(1, 178))
image_input = Input(shape=(64,64, 3))



In [52]:
cnn_output = CNN_model(image_input)
lstm_output = LSTM_model(eeg_input)

In [53]:
combined_output = Concatenate()([cnn_output, lstm_output])

In [54]:
x = Dense(64, activation='relu')(combined_output)
x = Dense(32, activation='relu')(x)
final_output = Dense(3, activation='softmax')(x) 

In [55]:
multi_head_model = Model(inputs=[eeg_input, image_input], outputs=final_output)

In [56]:
multi_head_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [57]:
multi_head_model.summary()

Model: "functional_69"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_16      │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_15      │ (None, 1, 178)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_4        │ (None, 128)       │ 67,120,960 │ input_layer_16[0… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_6        │ (None, 32)        │    704,384 │ input_layer_15[0… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 160)       │          0 │ sequential_4[0][… │
│ (Concatenate)       │                   │            │ sequential_6[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 64)        │     10,304 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_16 (Dense)    │ (None, 32)        │      2,080 │ dense_15[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_17 (Dense)    │ (None, 3)         │         99 │ dense_16[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 67,837,827 (258.78 MB)

 Trainable params: 67,837,123 (258.78 MB)

 Non-trainable params: 704 (2.75 KB)

In [58]:
history = multi_head_model.fit([X_train_signals, X_train_images], to_categorical(y_train),shuffle=False,validation_data=([X_test_signals,X_test_images],to_categorical(y_test)), epochs=10, batch_size=23)

Epoch 1/10
  62/1336 ━━━━━━━━━━━━━━━━━━━━ 19:01 896ms/step - accuracy: 0.3373 - loss: 0.6871

KeyboardInterrupt: 